# Validate On Anomalies

In [1]:
import numpy as np
import pandas as pd

import os
import sys
from pathlib import Path
SCRIPT_DIR = os.path.dirname(os.path.abspath("__init__.py"))
SRC_DIR = Path(SCRIPT_DIR).parent.absolute()
print(SRC_DIR)
sys.path.append(os.path.dirname(SRC_DIR))

from src.config import TA_MU, TA_SIGMA, SLP_MU, SLP_SIGMA

/Users/yannis/PycharmProjects/WeRec3D/src


# Compare Normalized Anomaly MAEs 

In [4]:
EXPERIMENTS = [
               # 'ex1_baseline',  # Is already anomaly 
               'ex2_seasonal_component',
               'ex3_incremental_pretraining',
               'ex3.1_moving_window',
               'ex3.2_cm_inclusion',
               'ex3.3_elevation',
               'ex3.4_pi_init',
               'ex4.1_elev_mov_win',
               'ex4.2_elev_cmi',
               'ex4.3_elev_pi_init',
               'ex5.1_elev_mov_cmi',
               'ex5.2_elev_mov_pi',
               'ex6.1_elev_mov_cmi_pi'
               ]

In [5]:
# Unit refers to °C or Pa
unit_val_ymean = np.load("../data_sets/ymean_sets/validation_ymean.npy")

normalized_gt = np.load("../data_sets/variant_b/validation.npy")
normalized_gt = normalized_gt[:3644, 6:38, 2:66]

normalized_val_ymean = (unit_val_ymean - np.array([TA_MU, SLP_MU])) / np.array([TA_SIGMA, SLP_SIGMA])
normalized_val_ymean = normalized_val_ymean[:3644]

exp_mae_map = {}
for exp_name in EXPERIMENTS:
    normalized_pred = np.load(f"{exp_name}/prediction/predicted.npy")
    
    norm_pred_anom = normalized_pred - normalized_val_ymean
    norm_gt_anom = normalized_gt - normalized_val_ymean
    
    normalized_anomaly_mae = np.mean(np.abs(norm_pred_anom - norm_gt_anom)) 
    print(f"Experiment {exp_name}: {normalized_anomaly_mae}")
    exp_mae_map[exp_name] = normalized_anomaly_mae

Experiment ex2_seasonal_component: 0.1653537646515857
Experiment ex3_incremental_pretraining: 0.11063940136725196
Experiment ex3.1_moving_window: 0.12439385664960434
Experiment ex3.2_cm_inclusion: 0.1138046809053312
Experiment ex3.3_elevation: 0.11059134599024154
Experiment ex3.4_pi_init: 0.11306821929033402
Experiment ex4.1_elev_mov_win: 0.10643644650633309
Experiment ex4.2_elev_cmi: 0.11329739505086453
Experiment ex4.3_elev_pi_init: 0.11422359082186109
Experiment ex5.1_elev_mov_cmi: 0.103159077336018
Experiment ex5.2_elev_mov_pi: 0.10840841704207617
Experiment ex6.1_elev_mov_cmi_pi: 0.10930415237565502


In [6]:
# Almost IDENTICAL to the climatology-MAE results! -> Seasonal component has been learned very well!
exp_mae_map

{'ex2_seasonal_component': 0.1653537646515857,
 'ex3_incremental_pretraining': 0.11063940136725196,
 'ex3.1_moving_window': 0.12439385664960434,
 'ex3.2_cm_inclusion': 0.1138046809053312,
 'ex3.3_elevation': 0.11059134599024154,
 'ex3.4_pi_init': 0.11306821929033402,
 'ex4.1_elev_mov_win': 0.10643644650633309,
 'ex4.2_elev_cmi': 0.11329739505086453,
 'ex4.3_elev_pi_init': 0.11422359082186109,
 'ex5.1_elev_mov_cmi': 0.103159077336018,
 'ex5.2_elev_mov_pi': 0.10840841704207617,
 'ex6.1_elev_mov_cmi_pi': 0.10930415237565502}

In [8]:
df = pd.DataFrame(data={'exp_name': exp_mae_map.keys(), 'anomaly_mae': exp_mae_map.values()} )
df.round(4)

,exp_name,anomaly_mae
0,ex2_seasonal_component,0.1654
1,ex3_incremental_pretraining,0.1106
2,ex3.1_moving_window,0.1244
3,ex3.2_cm_inclusion,0.1138
4,ex3.3_elevation,0.1106
5,ex3.4_pi_init,0.1131
6,ex4.1_elev_mov_win,0.1064
7,ex4.2_elev_cmi,0.1133
8,ex4.3_elev_pi_init,0.1142
9,ex5.1_elev_mov_cmi,0.1032
